### Sample program for Association Analysis (Market Basket Analysis) using FP-Growth  

#### Import libraries  

In [29]:
import pandas as pd
import pyfpgrowth  # https://fp-growth.readthedocs.io/en/latest/

#### Parameters  

In [30]:
csv_in = 'dm-12-quiz.csv'

#### Read CSV file  

In [31]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

(318, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 318 entries, 0 to 317
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   InvoiceNo  318 non-null    object
 1   ItemNo     318 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.1+ KB
None


,InvoiceNo,ItemNo
0,T001,5
1,T001,4
2,T001,6
3,T002,2
4,T003,2


#### Check appearance of each StockCode  

In [32]:
top_sc = df['ItemNo'].value_counts()
print(top_sc.size)
print(top_sc.head())

7
2    74
3    60
4    48
1    47
6    37
Name: ItemNo, dtype: int64


#### Format transaction data  
Before:  
```
invoice1, stockcode1, ...  
invoice1, stockcode2, ...  
invoice2, stockcode1, ...  
invoice3, stockcode1, ...  
invoice3, stockcode3, ...  
```

After:  
```
[  
  [stockcode1, stockcode2],    
  [stockcode1],  
  [stockcode1, stockcode3],  
  ...  
]  
```

and stockcode should be interger.  

#### Assign an integer (ID) to each stockcode  

In [33]:
# id2sc = sorted(list(set(df['InvoiceNo'])))
# sc2id = {}
# for i in range(len(id2sc)):
#     sc2id[id2sc[i]] = i

In [34]:
# df['Invoice_ID'] = df['InvoiceNo'].map(lambda x: sc2id[x])
# display(df.head())

#### Grouping transaction data by InvoiceNo  

In [35]:
invoices = []
for r in df.groupby('InvoiceNo'):
    s1 = set(r[1]['ItemNo'])
    invoices.append(list(s1))
print(len(invoices))
print(invoices[:3])

150
[[4, 5, 6], [2], [2, 3, 7]]


#### Market Basket Analysis by FP-Growth  

In [36]:
%time patterns = pyfpgrowth.find_frequent_patterns(invoices, 5)

CPU times: user 800 µs, sys: 0 ns, total: 800 µs
Wall time: 804 µs


In [37]:
#print(patterns)

In [38]:
%time rules = pyfpgrowth.generate_association_rules(patterns, 0.3)

CPU times: user 0 ns, sys: 58 µs, total: 58 µs
Wall time: 59.4 µs


In [39]:
print(rules)

{(1, 3): ((2,), 0.3125), (1, 2): ((4,), 0.35), (3,): ((2,), 0.3448275862068966), (2,): ((4,), 0.35135135135135137)}


In [40]:
results = []
for x in rules:
    ret = [x, rules[x][0], rules[x][1]]
    results.append(ret)
df_res = pd.DataFrame(results)
df_res.columns = ['LHS', 'RHS', 'Conf']

In [41]:
display(df_res.sort_values(by='Conf', ascending=False))

,LHS,RHS,Conf
3,"(2,)","(4,)",0.351351
1,"(1, 2)","(4,)",0.350000
2,"(3,)","(2,)",0.344828
0,"(1, 3)","(2,)",0.312500


#### Get original StockCode  

In [42]:
print(id2sc[1474])
print(id2sc[1475])
print(id2sc[1478])

IndexError: list index out of range

#### Calculation of Lift  

In [43]:
n_all = len(invoices)
lift = []
for i in range(df_res.shape[0]):
    rhs = df_res.at[i, 'RHS']
    conf = df_res.at[i, 'Conf']
    n_rhs = 0
    for items in invoices:
        if set(items) >= set(rhs):
            n_rhs += 1
    lift1 = conf / (n_rhs / n_all)
    lift.append(lift1)
    
df_res['Lift'] = lift

In [44]:
display(df_res.sort_values(by='Conf', ascending=False))

,LHS,RHS,Conf,Lift
3,"(2,)","(4,)",0.351351,1.097973
1,"(1, 2)","(4,)",0.350000,1.093750
2,"(3,)","(2,)",0.344828,0.698975
0,"(1, 3)","(2,)",0.312500,0.633446
